In [1]:
import torch
from torch import tensor
from torch.autograd import grad
import numpy as np

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from math import log10, floor, ceil, frexp

In [2]:
def mnk(x, y):
    b = (np.mean(x*y) - np.mean(x)*np.mean(y))/(np.mean(x**2) - np.mean(x)**2)
    a = np.mean(y) - b*np.mean(x)
    sb = 1/np.sqrt(len(x)) * np.sqrt((np.mean(y**2) - np.mean(y)**2)/(np.mean(x**2) - np.mean(x)**2) - b**2)
    sa = sb*np.sqrt(np.mean(x**2) - np.mean(x)**2)
    
    return a, b, sa, sb

def value(x, err):
    values_dict[x] = err
    
    x = tensor(x, requires_grad=True, dtype=torch.float64)
    all_values.append(x)
    all_errors.append(err)
    return x

def get_error(x):
    grads = grad(x, all_values, retain_graph=True, allow_unused=True)
    res = 0
    for i, error in enumerate(all_errors):
        if grads[i] is not None:
            res += (grads[i]*error)**2
    return res**0.5

def get_man_exp(x):
    e = floor(log10(x))
    m = x * 10**(-e)
    return m, e

def round2one(err):
    m, e = get_man_exp(err)
    m = round(m, 1)
    if int(m) in [1, 2]:
        return int(m*10), e - 1
    else:
        return int(m + 1), e
    
def science_notation(x, err):
    err, er = round2one(err)
    x = round(x, -er)
    m, e = get_man_exp(x)
    
    if abs(e) <= 1:
        print(f'${x} \\pm {err * 10 ** (er)}$')
    else:
        print(f'$({m} \\pm {err * 10 ** (-e+er)}) \\cdot 10^{"{" + str(e) + "}"}$')

def show(x, dx):
    x = float(x.detach())
    dx = float(dx)
    return science_notation(x, dx)

def mnk_a0(x, y):
    b = np.mean(x*y)/np.mean(x**2)
    sb = 1/np.sqrt(len(x)) * np.sqrt((np.mean(y**2)/np.mean(x**2)) - b**2)
    
    return b, sb

In [3]:
all_values = []
all_errors = []
values_dict = {}

df1 = pd.read_csv('data/1.txt')
df2 = pd.read_csv('data/2.txt')

M = value(2.9, 5/1000)
L = value(221/100, 0.5/100)
m1 = value(729.9/1000, 0.1/1000)
m2 = value(730.6/1000, 0.1/1000)
R = value(36/100, 0.4/100)
r = value(20.4/100, 0.4/100)
d = value(4.5/1000, 0.1/1000)
T1 = value(6.683, 0.2/11)
T2 = value(4.993, 0.2/11)
l = value(143/100, 0.5/100)
g = 9.8

ms1 = [value(t/1000, 0.0001/1000) for t in df1.m]
As1 = [value(t/1000, 0.25/1000) for t in df1.A]

ms2 = [value(t/1000, 0.0001/1000) for t in df2.m]
As2 = [value(t/100, 0.25/1000) for t in df2.A]

In [4]:
print(df2.style.to_latex())

\begin{tabular}{lrr}
 & m & A \\
0 & 0.506800 & 6.000000 \\
1 & 0.506700 & 5.800000 \\
2 & 0.512400 & 6.000000 \\
3 & 0.500700 & 6.200000 \\
4 & 0.506300 & 6.000000 \\
\end{tabular}



### Первый эксперимент

In [5]:
us = [M/ms1[i] * torch.sqrt(g/L) * As1[i] for i in range(len(ms1))]

In [6]:
for u in us:
    su = get_error(u)
    show(u, su)
    print(su/u)

$(1.35 \pm 0.04) \cdot 10^{2}$
tensor(0.0223, dtype=torch.float64, grad_fn=<DivBackward0>)
$(1.36 \pm 0.04) \cdot 10^{2}$
tensor(0.0223, dtype=torch.float64, grad_fn=<DivBackward0>)
$(1.36 \pm 0.04) \cdot 10^{2}$
tensor(0.0223, dtype=torch.float64, grad_fn=<DivBackward0>)
$(1.3900000000000001 \pm 0.04) \cdot 10^{2}$
tensor(0.0218, dtype=torch.float64, grad_fn=<DivBackward0>)
$(1.3800000000000001 \pm 0.04) \cdot 10^{2}$
tensor(0.0218, dtype=torch.float64, grad_fn=<DivBackward0>)


In [7]:
show(sum(us)/5, get_error(sum(us)/5))

$(1.368 \pm 0.014) \cdot 10^{2}$


In [8]:
Q = [ms1[i]*us[i]**2/2 * (1 - ms1[i]/M) for i in range(len(ms1))]
delta_T = [Q[i] / (140 * ms1[i]) for i in range(len(ms1))]
delta_T

[tensor(64.8741, dtype=torch.float64, grad_fn=<DivBackward0>),
 tensor(66.1660, dtype=torch.float64, grad_fn=<DivBackward0>),
 tensor(66.4290, dtype=torch.float64, grad_fn=<DivBackward0>),
 tensor(69.1668, dtype=torch.float64, grad_fn=<DivBackward0>),
 tensor(67.6301, dtype=torch.float64, grad_fn=<DivBackward0>)]

In [9]:
show(sum(Q)/len(Q), get_error(sum(Q)/len(Q)))

$4.74 \pm 0.1$


In [10]:
for q in Q:
    show(q, get_error(q))
    print(get_error(q)/q)

$4.63 \pm 0.21$
tensor(0.0446, dtype=torch.float64, grad_fn=<DivBackward0>)
$4.68 \pm 0.21$
tensor(0.0446, dtype=torch.float64, grad_fn=<DivBackward0>)
$4.68 \pm 0.21$
tensor(0.0446, dtype=torch.float64, grad_fn=<DivBackward0>)
$4.89 \pm 0.21$
tensor(0.0437, dtype=torch.float64, grad_fn=<DivBackward0>)
$4.83 \pm 0.21$
tensor(0.0437, dtype=torch.float64, grad_fn=<DivBackward0>)


In [11]:
for delta in delta_T:
    show(delta, get_error(delta))
    print(get_error(delta)/delta)

$64.9 \pm 2.9000000000000004$
tensor(0.0446, dtype=torch.float64, grad_fn=<DivBackward0>)
$66.0 \pm 4$
tensor(0.0446, dtype=torch.float64, grad_fn=<DivBackward0>)
$66.0 \pm 4$
tensor(0.0446, dtype=torch.float64, grad_fn=<DivBackward0>)
$69.0 \pm 4$
tensor(0.0437, dtype=torch.float64, grad_fn=<DivBackward0>)
$68.0 \pm 4$
tensor(0.0437, dtype=torch.float64, grad_fn=<DivBackward0>)


In [12]:
show(sum(delta_T)/len(delta_T), get_error(sum(delta_T)/len(delta_T)))

$66.9 \pm 1.3$


### Второй эксперимент

In [13]:
skI = 2*np.pi*((m1+m2)*R**2*T1)/(T1**2 - T2**2)
us2 = [skI/(ms2[i]*r) * (As2[i] / l) for i in range(len(ms2))]

In [14]:
show(skI, get_error(skI))

$0.403 \pm 0.01$


In [15]:
for u in us2:
    su = get_error(u)
    show(u, su)
    print(su/u)

$(1.6300000000000001 \pm 0.06) \cdot 10^{2}$
tensor(0.0329, dtype=torch.float64, grad_fn=<DivBackward0>)
$(1.58 \pm 0.06) \cdot 10^{2}$
tensor(0.0330, dtype=torch.float64, grad_fn=<DivBackward0>)
$(1.62 \pm 0.06) \cdot 10^{2}$
tensor(0.0329, dtype=torch.float64, grad_fn=<DivBackward0>)
$(1.71 \pm 0.06) \cdot 10^{2}$
tensor(0.0329, dtype=torch.float64, grad_fn=<DivBackward0>)
$(1.6400000000000001 \pm 0.06) \cdot 10^{2}$
tensor(0.0329, dtype=torch.float64, grad_fn=<DivBackward0>)
